## 1.
Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД.  

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
from pymongo import MongoClient
from pprint import pprint 
import pandas as pd

In [2]:
client = MongoClient('127.0.0.1', 27017)
db = client['parser_vacancies']
vacancies = db.parser_vacancies

In [3]:
vacancies

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'parser_vacancies'), 'parser_vacancies')

In [4]:
job = input()

product


In [5]:
main_link = 'https://hh.ru'
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) \
        Chrome/89.0.4389.90 Safari/537.36'}
link = f'{main_link}/search/vacancy/'
params = {'L_save_area': 'true', 'clusters': 'true','enable_snippets': 'true', 'text': {job}, 'showClusters': 'true'}

In [6]:
def insert_jobs(data):
    vacancies.insert_one({
        'vacancy_title': data['title'],
        'link': data['link'],
        'city': data['city'],
        'company': data['company_name'],
        'metro': data['metro_station'], 
        'salary_min': data['salary_min'],
        'salary_max': data['salary_max'],
        'salary_currency': data['salary_currency']})

In [7]:
result = []
while True:
    response = requests.get(link, headers=header, params=params)
    soup = bs(response.text, 'html.parser')

    if response.ok:    
        vacancy_list = soup.find_all('div', {'class': 'vacancy-serp-item'})
    
        for i in vacancy_list:
            data = {}    
            
            #title
            vacancy_title = i.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
            title = vacancy_title.text
            data['title'] = title
    
            #vacancy url_link
            data['link'] = vacancy_title['href']
    
            #city
            data['city'] = i.find('span', {'data-qa': 'vacancy-serp__vacancy-address'}).text
    
            #company name
            company_name = i.find('div', {'class': 'vacancy-serp-item__meta-info'}).find('a').getText() \
            .replace(u'\xa0', u'')    
            data['company_name'] = company_name
    
            #metro
            metro_station = i.find('span', {'class': 'vacancy-serp-item__meta-info'}).findChild()
            if not metro_station:
                metro_station = None
            else:
                metro_station = metro_station.getText()        
            data['metro_station'] = metro_station 
    
            #salary
            vacancy_salary = i.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})    
            vacancy_min = None
            vacancy_max = None
            vacancy_currency = None
        
            if vacancy_salary is not None:
                vacancy_salary = vacancy_salary.getText().replace('\u202f','')
                vacancy_currency = vacancy_salary.split()[-1]
            
                if vacancy_salary.find('–') > -1:
                    vacancy_min = int(vacancy_salary.split()[0])
                    vacancy_max = int(vacancy_salary.split()[2])
                elif vacancy_salary.find('от'):
                    vacancy_min = int(vacancy_salary.split()[1])
                elif vacancy_salary.find('до'):
                    vacancy_max = int(vacancy_salary.split()[1])
        
            data['salary_min'] = vacancy_min
            data['salary_max'] = vacancy_max
            data['salary_currency'] = vacancy_currency
    
            result.append(data) 
            insert_jobs(data)
            next_page = soup.find('a', {'data-qa': 'pager-next'})
            
        if next_page is None:
            break
        params = next_page['href']

In [8]:
# for v in vacancies.find({}):
#     pprint(v)

## 2.
Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы. Поиск должен происходить по 2-ум полям (минимальной и максимальной зарплате).  

In [9]:
salary = int(input())

200000


In [10]:
def check_salary(salary):     
    for i in vacancies.find({'$or': [{'salary_min':{'$gte': salary}}, {'salary_max':{'$gte': salary}}]}).sort('salary_min'):
        pprint(i)

In [11]:
# check_salary(salary)

## 3.
Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.

In [12]:
def add_new_jobs():
    a = 0
    for i in result:
        link_new = i['link']
        if vacancies.count_documents({'link': link_new}) == 0:
            vacancies.insert_one(i)
            a += 1
            
    return a

In [13]:
add_new_jobs()

0